# **PHOC Model Implementation**

This notebook is an implementation of the PHOCNet architecture for our Cropped Word Recognition dataset and task

### Directory Manager
- To manage the different required file paths

In [18]:
# Save the generated trained model
saved_model_phocnet = 'PATH'

# Lexicon file (file containing necessary words -> knn, data generation...)
lexicon_file = 'PATH' 

# Bigrams for phoc
bigrams_file = 'PATH' 

# Access the trained KNN Classifier
store_knn_classifier = 'PATH'

# Image CSV files (image_path/label) in this case our generated dataset
train_images_csv = 'PATH'
test_images_csv = 'PATH'

In [ ]:
# Libraries 
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torchvision.io import read_image

import numpy as np
import os

import pickle
from sklearn.neighbors import KNeighborsClassifier

from torchvision import transforms
from torch.optim.lr_scheduler import StepLR, CyclicLR, CosineAnnealingLR, ReduceLROnPlateau
from torchvision import datasets, models, transforms


!pip install editdistance
import editdistance

## Model + SPP

- PHOCNet pipeline
- SPP to use any input size images

In [19]:
# Spatial pyramid pooling implementation
class SPP(nn.Module):

    def __init__(self, levels = 3, pool_type = 'max_pool'):
        super(SPP, self).__init__()

        if pool_type not in ['max_pool', 'avg_pool', 'max_avg_pool']:
            raise ValueError('Unknown pool_type. Must be either \'max_pool\', \'avg_pool\' or both')
        
        self.pooling_output_size = sum([4 ** level for level in range(levels)]) * 512

        self.levels = levels
        self.pool_type = pool_type

    def forward(self, input_x):
        out = self._spatial_pyramid_pooling(input_x, self.levels)
        return out
    
    def _pyramid_pooling(self, input_x, output_sizes):
        pyramid_level_tensors = []
        for tsize in output_sizes:
            if self.pool_type == 'max_pool':
                pyramid_level_tensor = F.adaptive_max_pool2d(input_x, tsize)
                pyramid_level_tensor = pyramid_level_tensor.view(input_x.size(0), -1)
            if self.pool_type == 'avg_pool':
                pyramid_level_tensor = F.adaptive_avg_pool2d(input_x, tsize)
                pyramid_level_tensor = pyramid_level_tensor.view(input_x.size(0), -1)
            if self.pool_type == 'max_avg_pool':
                pyramid_level_tensor_max = F.adaptive_max_pool2d(input_x, tsize)
                pyramid_level_tensor_max = pyramid_level_tensor_max.view(input_x.size(0), -1)
                pyramid_level_tensor_avg = F.adaptive_avg_pool2d(input_x, tsize)
                pyramid_level_tensor_avg = pyramid_level_tensor_avg.view(input_x.size(0), -1)
                pyramid_level_tensor = torch.cat([pyramid_level_tensor_max, pyramid_level_tensor_avg], dim=1)

            pyramid_level_tensors.append(pyramid_level_tensor)

        return torch.cat(pyramid_level_tensors, dim=1)

    def _spatial_pyramid_pooling(self, input_x, levels):
        output_sizes = [(int( 2 **level), int( 2 **level)) for level in range(levels)]
        return self._pyramid_pooling(input_x, output_sizes)


# PHOCnet architecture
class PHOCNet(nn.Module):

    def __init__(self, n_out, input_channels = 3, pooling_levels = 3, pool_type = 'max_pool'):
        super(PHOCNet, self).__init__()

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        self.conv_block3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        self.conv_block4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )
        
        self.pooling_layer_fn = SPP(levels = pooling_levels, pool_type=pool_type)
        pooling_output_size = self.pooling_layer_fn.pooling_output_size
        
        self.fc1 = nn.Linear(pooling_output_size, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, n_out)

    
    # Forward function (organized by blocks as the implementation)
    def forward(self, x):
        out = self.conv_block1(x)
        out = F.max_pool2d(out, kernel_size=2, stride=2, padding=0)
        out = self.conv_block2(out)
        out = F.max_pool2d(out, kernel_size=2, stride=2, padding=0)
        out = self.conv_block3(out)
        out = self.conv_block4(out)

        out = self.pooling_layer_fn(out)
        out = self.fc1(out)
        out = F.relu(out)
        out = F.dropout(out, p = 0.5, training = self.training)
        out = self.fc2(out)
        out = F.relu(out)
        out = F.dropout(out, p = 0.5, training = self.training)
        out = self.fc3(out)

        return out

## PHOC Vector functions

- Functions to obtain PHOC vector of any string of characters

In [20]:
def build_phoc(words, phoc_unigrams, unigram_levels,
               bigram_levels=None, phoc_bigrams=None,
               split_character=None):
    '''
    Calculate Pyramidal Histogram of Characters (PHOC) descriptor
    
        word (str): word to calculate descriptor for
        phoc_unigrams (str): string of all unigrams to use in the PHOC
        unigram_levels (list of int): the levels for the unigrams in PHOC
        phoc_bigrams (list of str): list of bigrams to be used in the PHOC
        phoc_bigram_levls (list of int): the levels of the bigrams in the PHOC
        split_character (str): special character to split the word strings into characters
        on_unknown_unigram (str): What to do if a unigram appearing in a word
            is not among the supplied phoc_unigrams. Possible: 'warn', 'error'

    Returns
        the PHOC for the given word
    '''
    # prepare output matrix
    phoc_size = len(phoc_unigrams) * np.sum(unigram_levels)
    if phoc_bigrams is not None:
        phoc_size += len(phoc_bigrams) * np.sum(bigram_levels)
    phocs = np.zeros((len(words), phoc_size))

    # prepare some lambda functions
    occupancy = lambda k, n: [float(k) / n, float(k + 1) / n]
    overlap = lambda a, b: [max(a[0], b[0]), min(a[1], b[1])]
    size = lambda region: region[1] - region[0]

    # map from character to alphabet position
    char_indices = {d: i for i, d in enumerate(phoc_unigrams)}

    # iterate through all the words
    for word_index, word in enumerate(words):
        """if '0' in word or '1' in word or '2' in word or '3' in word or '4' in word or '5' in word or '6' in word or '7' in word or '7' in word or '8' in word or '9' in word:
            continue"""
        if split_character is not None:
            word = word.split(split_character)
        n = len(word)
        for index, char in enumerate(word):
            char_occ = occupancy(index, n)        
            char_index = char_indices[char]

            for level in unigram_levels:
                for region in range(level):
                    region_occ = occupancy(region, level)
                    if size(overlap(char_occ, region_occ)) / size(char_occ) >= 0.5:
                        feat_vec_index = sum([l for l in unigram_levels if l < level]) * len(
                            phoc_unigrams) + region * len(phoc_unigrams) + char_index
                        phocs[word_index, feat_vec_index] = 1

        # add bigrams
        if phoc_bigrams is not None:
            ngram_features = np.zeros(len(phoc_bigrams) * np.sum(bigram_levels))
            ngram_occupancy = lambda k, n: [float(k) / n, float(k + 2) / n]

            for i in range(n - 1):
                ngram = word[i:i + 2]
                phoc_dict = {k: v for v, k in enumerate(phoc_bigrams)}
                if phoc_dict.get(ngram, 666) == 666:
                    continue
                occ = ngram_occupancy(i, n)

                for level in bigram_levels:
                    for region in range(level):
                        region_occ = occupancy(region, level)
                        overlap_size = size(overlap(occ, region_occ)) / size(occ)
                        if overlap_size >= 0.5:
                            ngram_features[region * len(phoc_bigrams) + phoc_dict[ngram]] = 1
            phocs[word_index, -ngram_features.shape[0]:] = ngram_features

    return phocs

def phoc(raw_word):
    '''

    :param raw_word: string of word to be converted
    :return: phoc representation as a np.array (1,604)
    '''
    if type(raw_word) == type([]):
        word = [w.lower() for w in raw_word]
    else:
        word =[raw_word]
        word_lowercase = word[0].lower()
        word = [word_lowercase]
    phoc_unigrams = '0123456789abcdefghijklmnopqrstuvwxyz'
    unigram_levels = [2,3,4,5]
    bigram_levels=[]
    bigram_levels.append(2)
    
    phoc_bigrams = []
    i = 0
    with open(bigrams_file,'r') as f:
        for line in f:
            a = line.split()
            phoc_bigrams.append(a[0].lower())
            i = i +1
            if i >= 50:break

    
    qry_phocs = build_phoc(words = word, phoc_unigrams = phoc_unigrams, unigram_levels = unigram_levels,
                           bigram_levels = bigram_levels, phoc_bigrams = phoc_bigrams)
    qry_phocs = build_phoc(words = word, phoc_unigrams = phoc_unigrams, unigram_levels = unigram_levels)
    
    return qry_phocs

## KNN Generator

- To generate a KNN as it effectively classifies character histograms by the local neighborhood information, enhancing text recognition accuracy
- No need to run again if the KNN Classifier has already been created

In [21]:
with open(lexicon_file, "r") as file:
    list_of_words = file.readlines()

list_of_words = [l[:-1] for l in list_of_words]
phoc_representations = phoc(list_of_words)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(phoc_representations, list_of_words)

knnPickle = open(store_knn_classifier, 'wb') 
pickle.dump(knn, knnPickle)  
knnPickle.close()

'with open(lexicon_file, "r") as file:\n    list_of_words = file.readlines()\n\n\n"""img_dir = ""\nlist_of_words = os.listdir(img_dir)\nlist_of_words = [word.split("_")[-1].split(".")[0] for word in list_of_words]"""\nlist_of_words = [l[:-1] for l in list_of_words]\nphoc_representations = phoc(list_of_words)\n\nknn = KNeighborsClassifier(n_neighbors=1)\nknn.fit(phoc_representations, list_of_words)\n\nknnPickle = open(store_knn_classifier, \'wb\') \npickle.dump(knn, knnPickle)  \nknnPickle.close()'

## Dataset 

- Pass images, their PHOC-Vector label and the label in string format
- Image transforms as well

In [22]:
class dataset(Dataset):
    def __init__(self, img_dir, transform = None):
        
        self.paths = os.listdir(img_dir)
        #self.paths = self.paths[:int(len(self.paths)*0.01)]
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):

        return len(self.paths)
    
    def __getitem__(self, idx):

        path = self.img_dir + self.paths[idx]

        img = read_image(path)
        img = img.to(torch.float32)
        if self.transform != None:
            img = self.transform(img)
        img /= 255
        
        word = self.paths[idx].split("_")[-1].split(".")[0]
        target = phoc(word)
        target = target.reshape(target.shape[1])

        return img, target, word

## Utility functions

- load_model() -> load the KNN model

- predict_with_PHOC() -> predict a label using the PHOC model
- get_data() -> create a custom pytorch 'dataset' from an image path 
- make_loader() -> loads the created 'dataset' using a pytorch dataloader
- make() -> initialize the model with all of its attritbutes, data, optimizer, scheduler criterion & weights 
- init_weights_model() -> initiates the models for the weights using a kaiming normal distribution
- create_weights() -> creates a tensor for the weights in the training section: the idea is to define the weights for an error as the inverse of the 'appeareance frequency' of the that word in the lexicon.txt file
- set_parameter_requires_grad() -> freezes the parameters of a neural network model, preventing their gradients from being computed during training, when the feature_extracting flag is set to True

In [24]:
def load_model():
    # Load the pre-trained KNN model from a file
    return pickle.load(open(store_knn_classifier, 'rb'))

def predict_with_PHOC(phocs, model):
    # Predict labels using the KNN model with PHOC representations
    result = model.predict(phocs)
    return result

def get_data(img_dir, transform=None):
    # Create a custom dataset from the image directory with optional transformations
    created_dataset = dataset(img_dir, transform)
    return created_dataset

def make_loader(dataset, batch_size):
    # Create a DataLoader for the dataset with the specified batch size and other parameters
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         pin_memory=True, num_workers=2)
    return loader

def make(config, device="cuda"):
    # Initialize the model, data loaders, optimizer, scheduler, and loss function
    print("Starting the make function...")

    transforms_train = transforms.Compose([
        transforms.Resize((64, 128), antialias=True),
        transforms.Normalize(mean=[0.445313568], std=[0.26924618])
    ])
    print("Training transforms created.")

    transforms_test = transforms.Compose([
        transforms.Resize((64, 128), antialias=True),
        transforms.Normalize(mean=[0.445313568], std=[0.26924618])
    ])
    print("Testing transforms created.")

    train, test = get_data(config.train_dir, transforms_train), get_data(config.test_dir, transforms_test)
    print(f"Training data and testing data loaded. Number of training samples: {len(train)}, Number of testing samples: {len(test)}")

    train_loader = make_loader(train, config.batch_size)
    test_loader = make_loader(test, config.batch_size)
    print(f"Data loaders created. Batch size: {config.batch_size}")

    # Make the model
    model = PHOCNet(n_out=train[0][1].shape[0], input_channels=1).to(device)
    model.apply(init_weights_model)
    print("Model created and weights initialized.")

    pos_weight = torch.tensor(create_weights(config.train_dir)).to(device)
    criterion = nn.BCEWithLogitsLoss(reduction='mean', pos_weight=pos_weight)
    print(f"Loss function created with pos_weight: {pos_weight}")

    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    print(f"Optimizer created with learning rate: {config.learning_rate}")

    scheduler = ReduceLROnPlateau(optimizer, patience=2, factor=0.1)
    print("Scheduler created.")

    print("Make function completed.")
    return model, train_loader, test_loader, criterion, optimizer, scheduler

def init_weights_model(m):
    # Initialize weights of Conv2d and Linear layers with Kaiming normal distribution
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

def create_weights(file_words):
    # Create weights for the loss function based on the frequency of words in the dataset
    paths = os.listdir(file_words)
    list_of_words = [path.split("_")[-1].split(".")[0] for path in paths]
    list_of_words = [l[:-1] for l in list_of_words]
    phoc_representations = phoc(list_of_words)
    suma = np.sum(phoc_representations, axis=0)
    weights = phoc_representations.shape[0]/(suma+1e-6) 
    weights = (1 + (weights/max(weights)))*5
    return weights

def set_parameter_requires_grad(model, feature_extracting):
    # Freeze parameters of the model if feature_extracting is True
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

## Wandb functions

- To keep track of our execution data on wandb and draw images with their corresponding labels

In [25]:
# Install the wandb library for experiment tracking and logging
!pip install wandb

import wandb
from torchvision import transforms as T
from PIL import ImageDraw, ImageFont  # Using PIL to draw labels on the images
from torchvision import transforms

def train_log(loss, total_example_ct):
    # Log training loss to wandb and print the current loss
    wandb.log({"loss": loss}, step=total_example_ct)
    print(f"Loss after {str(total_example_ct).zfill(5)} examples: {loss:.3f}")

def train_test_log(loss_test, loss_train, accuracy_test, accuracy_train, edit_test, edit_train, epoch):
    # Log training and testing metrics to wandb and print the losses
    wandb.log({
        "Epoch": epoch, 
        "Train loss": loss_train, "Test loss": loss_test,
        "Train accuracy": accuracy_train, "Test accuracy": accuracy_test,
        "Train edit": edit_train, "Test edit": edit_test,
    })
    print(f"Train Loss: {loss_train:.3f}\nTest Loss: {loss_test:.3f}")

def log_images(images, predicted_labels, text_labels, epoch, mode):
    # Log images with predicted and true labels to wandb
    t = transforms.Compose([transforms.Normalize(0, 1/0.1), transforms.Normalize(-0.5, 1)])
    t_images = t(images)
    images_with_labels = draw_images(t_images, text_labels, predicted_labels)
    wandb.log({f"Epoch{epoch}-{mode}": [wandb.Image(im) for im in images_with_labels]})

def draw_images(images, text_labels, predicted_labels):
    # Convert images to PIL format and draw labels on them
    transform = T.ToPILImage()
    images = [draw_one_image(transform(im), t_lab, p_lab) for im, t_lab, p_lab in zip(images, text_labels, predicted_labels)]
    return images

def draw_one_image(image, text_label, predicted_label):
    # Draw text labels on a single image and color code based on correctness
    image = image.convert("RGB")
    draw = ImageDraw.Draw(image)
    color = "green" if text_label == predicted_label else "red"
    text = text_label + "\n" + predicted_label
    font = ImageFont.truetype(f'/home/xnmaster/DlProject/deep-learning-project-2024-ai_nndl_group_01_/Generate Dataset/Fonts/calibri.ttf', 10)
    draw.text((0, 0), text, font=font, fill=color)
    return image

def lr_log(lr, epoch):
    # Log the learning rate to wandb
    wandb.log({"learning-rate": lr}, step=epoch)


## Train

In [26]:
def train(model, train_loader, test_loader, criterion, optimizer, scheduler, config, device = "cuda"):
    # Tell wandb to watch what the model gets up to: gradients, weights...
    wandb.watch(model, criterion, log="all", log_freq=10)

    example_ct = 0  # number of examples seen
    batch_ct = 0 
    model_phoc = load_model() # Loading the KNN model
    for epoch in (range(config.epochs)):
        model.train()
        train_loss = 0
        for _, (images, phoc_labels, _) in enumerate(train_loader):

            loss = train_batch(images, phoc_labels, model, optimizer, criterion, device)
            train_loss += loss.item()
            example_ct +=  len(images)
            batch_ct += 1

            # Report metrics every 25 batches
            if ((batch_ct + 1) % 25) == 0:
                train_log(loss.item(), example_ct)
        
        loss_test = test(model, test_loader, train_loader, epoch, criterion, model_phoc, device)
        
        scheduler.step(loss_test)
        print(scheduler._last_lr)
        torch.save(model.state_dict(), os.path.join(config.save_model, f"PHOCNET{epoch}.pt"))
    return model


# Train on individual batch
def train_batch(images, labels, model, optimizer, criterion, device="cuda"): # GPU if possible
    images, labels = images.to(device), labels.to(device)
    
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs.float(), labels.float())
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()

    # Optim step
    optimizer.step()

    return loss

## Test 

In [27]:
def test(model, test_loader, train_loader, epoch, criterion, model_phoc, device="cuda", save: bool = True):
    # Set the model to evaluation mode
    model.eval()
    with torch.no_grad():
        # Initialize variables to track losses, correct predictions, and edit distances
        loss_test = 0
        loss_train = 0
        correct_test = 0
        correct_train = 0
        edit_test = 0
        edit_train = 0
        test_count = 0
        train_count = 0
        
        # Iterate over the test data
        for i, (images, phoc_labels, text_labels) in enumerate(test_loader):
            images, phoc_labels = images.to(device), phoc_labels.to(device)
            test_count += len(images)
            outputs = model(images)
            loss_test += criterion(outputs, phoc_labels.float())
            predicted_labels = predict_with_PHOC(torch.sigmoid(outputs).cpu().numpy(), model_phoc)
            correct_test += (predicted_labels == text_labels).sum().item()
            edit_test += sum([editdistance.eval(p, t) for p, t in zip(predicted_labels, text_labels)])
            if i == 0:
                log_images(images, predicted_labels, text_labels[:5], epoch, "Test")
                
        # Iterate over the training data
        for i, (images, phoc_labels, text_labels) in enumerate(train_loader):
            images, phoc_labels = images.to(device), phoc_labels.to(device)
            train_count += len(images)
            outputs = model(images)
            loss_train += criterion(outputs, phoc_labels.float())
            predicted_labels = predict_with_PHOC(torch.sigmoid(outputs).cpu().numpy(), model_phoc)
            correct_train += (predicted_labels == text_labels).sum().item()
            edit_train += sum([editdistance.eval(p, t) for p, t in zip(predicted_labels, text_labels)])
            if i == 0:
                log_images(images, predicted_labels, text_labels[:5], epoch, "Train")
            if i == 150:
                break

        # Calculate average losses, accuracies, and edit distances
        loss_test = loss_test / len(test_loader)
        loss_train = loss_train / (i + 1)
        accuracy_test = correct_test / test_count
        accuracy_train = correct_train / train_count
        edit_test = edit_test / test_count
        edit_train = edit_train / train_count

        # Log training and testing metrics
        train_test_log(loss_test, loss_train, accuracy_test, accuracy_train, edit_test, edit_train, epoch)
    
    # Return the test loss
    return loss_test

## Main

In [28]:
# Ensure GPU usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Caller function to train the model
def model_pipeline(cfg:dict) -> None:
    with wandb.init(project="phoc-def", config=cfg): # Set up of project
        config = wandb.config

        model, train_loader, test_loader, criterion, optimizer, scheduler = make(config, device)
        model = train(model, train_loader, test_loader, criterion, optimizer, scheduler, config, device)

        return model

# Actual main, with wandb implementation
if __name__ == "__main__":
    wandb.login()

    config = dict(
        train_dir='ADD PATH'+"/",
        test_dir='ADD PATH'+"/",
        epochs=8,
        batch_size= 8,
        learning_rate=0.0001,
        save_model = saved_model_phocnet+"/")
    model = model_pipeline(config)      

cuda


Starting the make function...
Training transforms created.
Testing transforms created.
Training data and testing data loaded. Number of training samples: 50000, Number of testing samples: 5000
Data loaders created. Batch size: 8
Model created and weights initialized.
Loss function created with pos_weight: tensor([ 5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,
         5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,
         5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,
         5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,
         5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,
         5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,
         5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,
         5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,  5.0000,
         5.0000,  5.0000,  5.0000,  5.0000,  5.0000, 

Epoch,▁▂▃▄▅▆▇█
Test accuracy,▁▆▇▇█▇██
Test edit,█▃▂▂▁▂▁▁
Test loss,█▄▃▂▂▂▁▁
Train accuracy,▁▂▆▇▇█▇█
Train edit,█▆▃▂▂▁▂▁
Train loss,█▄▃▂▂▂▁▁
loss,██▆▃▂▃▂▂▂▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,7
Test accuracy,0.9978
Test edit,0.0028
